## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value
cutoff_value = 500  # You can adjust this cutoff value as needed

# Get the value counts of application types
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Identify application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index

# Replace in the DataFrame
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other", inplace=True)

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts=application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

In [8]:
# Choose a cutoff value (e.g., 1000) for classification counts
cutoff_value = 1000

# Get the counts of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced based on the cutoff value
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`

df = pd.DataFrame(application_df)

# Convert 'Category' column to dummy variables
df = pd.get_dummies(df).astype(int)

In [10]:
# Split our preprocessed data into our features and target arrays
# Assuming you have a DataFrame 'application_df' with preprocessed data

# Define your features (X) and target (y)
X = df.drop(columns=['IS_SUCCESSFUL'],axis=1)
y = df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=70)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

#### 1st trail 

In [12]:


# Load your dataset and preprocess it as needed
# X_train, y_train, X_val, y_val, X_test, y_test = ...
# nn_model = tf.keras.models.Sequential()
# Define the neural network model
nn_model = tf.keras.models.Sequential()

# Hidden layers
nn_model.add(Dense(units=80, activation='relu', input_dim=43))
nn_model.add(Dense(units=30, activation='relu')) # Fully connected layer with ReLU activation
nn_model.add(Dense(units=1, activation='sigmoid'))  # Fully connected layer with ReLU activation
# Output layer
# nn_model.add(10, activation='softmax')  # Output layer with softmax activation (e.g., for classification)


# Compile the model with different optimization methods
import tensorflow as tf

# Define the optimizers
optimizers = {
    'sgd': tf.keras.optimizers.SGD(),
    'Adam': tf.keras.optimizers.Adam(),
    'RMSprop': tf.keras.optimizers.RMSprop()
}

# Display the summary of nn_model before training
nn_model.summary()

# Iterate through different optimizers and train the model
for optimizer_name, optimizer in optimizers.items():
    nn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # Train the model
    fit_model = nn_model.fit(X_train_scaled, y_train, epochs=10)
    # Evaluate the model on the test data
    model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
    print("--------------------------")
    print(f"Optimizer: {optimizer_name}")
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    print("--------------------------")



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
804/804 [==============================] - 1s 654us/step - loss: 0.6156 - accuracy: 0.6970
Epoch 2/10
804/804 [==============================] - 0s 608us/step - loss: 0.5668 - accuracy: 0.7269
Epoch 3/10
804/804 [==============================] - 1s 623us/step - loss: 0.5603

#### Second trail with different number of neurons

In [13]:


# Load your dataset and preprocess it as needed
# X_train, y_train, X_val, y_val, X_test, y_test = ...
# nn_model = tf.keras.models.Sequential()
# Define the neural network model
nn_model2 = tf.keras.models.Sequential()

# Hidden layers
nn_model2.add(Dense(units=150, activation='relu', input_dim=43))
nn_model2.add(Dense(units=100, activation='relu')) # Fully connected layer with ReLU activation
nn_model2.add(Dense(units=1, activation='sigmoid'))  # Fully connected layer with ReLU activation
# Output layer
# nn_model.add(10, activation='softmax')  # Output layer with softmax activation (e.g., for classification)


# Compile the model with different optimization methods
import tensorflow as tf

# Define the optimizers
optimizers = {
    'sgd': tf.keras.optimizers.SGD(),
    'Adam': tf.keras.optimizers.Adam(),
    'RMSprop': tf.keras.optimizers.RMSprop()
}

# Display the summary of nn_model before training
nn_model2.summary()

# Iterate through different optimizers and train the model
for optimizer_name, optimizer in optimizers.items():
    nn_model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # Train the model
    fit_model = nn_model.fit(X_train_scaled, y_train, epochs=10)
    # Evaluate the model on the test data
    model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
    print("--------------------------")
    print(f"Optimizer: {optimizer_name}")
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    print("--------------------------")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 150)               6600      
                                                                 
 dense_4 (Dense)             (None, 100)               15100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21801 (85.16 KB)
Trainable params: 21801 (85.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
  1/804 [..............................] - ETA: 1s - loss: 0.5849 - accuracy: 0.6562

804/804 [==============================] - 1s 700us/step - loss: 0.5467 - accuracy: 0.7353
Epoch 2/10
804/804 [==============================] - 1s 644us/step - loss: 0.5464 - accuracy: 0.7356
Epoch 3/10
804/804 [==============================] - 1s 735us/step - loss: 0.5462 - accuracy: 0.7363
Epoch 4/10
804/804 [==============================] - 1s 699us/step - loss: 0.5470 - accuracy: 0.7364
Epoch 5/10
804/804 [==============================] - 1s 648us/step - loss: 0.5462 - accuracy: 0.7350
Epoch 6/10
804/804 [==============================] - 1s 761us/step - loss: 0.5459 - accuracy: 0.7373
Epoch 7/10
804/804 [==============================] - 1s 658us/step - loss: 0.5461 - accuracy: 0.7371
Epoch 8/10
804/804 [==============================] - 1s 758us/step - loss: 0.5460 - accuracy: 0.7367
Epoch 9/10
804/804 [==============================] - 1s 668us/step - loss: 0.5459 - accuracy: 0.7360
Epoch 10/10
804/804 [==============================] - 1s 736us/step - loss: 0.5454 - accurac

In [14]:
# # Export our model to HDF5 file
# nn_model.save("AlphabetSoupCharity.h5")